In [1]:
import pandas as pd
import numpy as np

import data_preprocessing
from data_preprocessing import DataCollector

import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import lxml
import pytesseract
import cv2
from PIL import Image
from io import BytesIO
from pathlib import Path

import matplotlib.pyplot as plt

import warnings 
from dadata import Dadata
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [ ]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.5.715 Yowser/2.5 Safari/537.36'
}

brands = DataCollector().parse_rospatent(headers=headers)

In [ ]:
brand_df = brands.read_data(zip_file=False).get_data()

In [4]:
brand_df.columns = brand_df.columns.map(lambda x: x.replace(' ', '_').lower()) 
brand_df = brand_df[['registration_number', 'registration_date', 'application_number', 'application_date',
                     'right_holder_inn', 'right_holder_ogrn', 'right_holder_name', 'publication_url']]

__Converting a data to proper datatype__

In [46]:
def to_int(cell):
    try:
        cell = int(cell)
    except:
        cell = 0
    return cell

def to_date_pattern(cell):
    cell = str(cell)
    if cell == 'nan':
        return cell
    else:
        return cell[:4] + '/' + cell[4:6] + '/' + cell[6:8]
    
def get_brand_name(url):
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.5.715 Yowser/2.5 Safari/537.36'
    }
    
    try:
        query = requests.get(url=url, headers=headers)
        content = query.content

        soup = BeautifulSoup(content, 'lxml')
        img_href = soup.find(id='mainDoc2').find_all('a', href=True)[-1]['href']

        img = requests.get(url=img_href, headers=headers).content
        image = Image.open(BytesIO(img))

        pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

        string = pytesseract.image_to_string(image, lang='rus')
    except:
        string = ''
    
    return string[:-1]

In [6]:
brand_df[['registration_number', 'application_number','right_holder_inn', 'right_holder_ogrn']] =\
    brand_df[['registration_number', 'application_number','right_holder_inn', 'right_holder_ogrn']].applymap(to_int)

brand_df[['registration_date', 'application_date']] =\
brand_df[['registration_date', 'application_date']].applymap(to_date_pattern)

brand_df = brand_df.rename(columns={'right_holder_ogrn': 'ogrn'})

__Не стал изобретать велосипед и спёр информацию о юр.лицах для примера у вас же)__

In [42]:
token = "ffaae93e81193382cf4c3ea7bd54da8bce35cb14"
dadata = Dadata(token)

raw_data = []
company_list = ['дейта кью', 'СФЕРА', 'Союз Святого Иоанна Воина']
for company in company_list:
    result = dadata.suggest("party", company)
    raw_data.append(result[0])

In [43]:
dadata = pd.DataFrame(pd.DataFrame(raw_data).data.to_list())
dadata['company_name'] = pd.DataFrame(raw_data).iloc[:,0]
dadata = dadata[['company_name', 'ogrn']]
dadata['ogrn'] = dadata['ogrn'].apply(to_int)
dadata = brand_df.merge(dadata, on='ogrn')

__Применяю функцию для парсинга картинок и обработки их tesserract__

In [47]:
dadata['brand_name'] = dadata.publication_url.apply(get_img)

In [50]:
dadata[dadata.brand_name != '']

,registration_number,registration_date,application_number,application_date,right_holder_inn,ogrn,right_holder_name,publication_url,company_name,brand_name
63,701251,2019/03/01,2018728690,2018/07/09,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=701251,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",BePHbIN\n
66,725407,2019/09/02,2018728662,2018/07/09,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=725407,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",« B@PHbIM yHucepcam\n
67,725408,2019/09/02,2018728665,2018/07/09,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=725408,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",yHueepcam BEPHbIM »»»\n
68,725409,2019/09/02,2018728676,2018/07/09,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=725409,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",yHueepcam B@PHbIN »»\n
69,725410,2019/09/02,2018728679,2018/07/09,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=725410,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",« BEPHbIN yHusepcam\n\n
...,...,...,...,...,...,...,...,...,...,...
187,907225,2022/12/02,2022741212,2022/06/23,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=907225,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",START UP\n
189,912617,2022/12/20,2021747097,2021/07/27,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=912617,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",BUYJUIA JIMWA\nVILLA LIDIA\n
190,913526,2022/12/30,2022767348,2022/09/21,7729705354,1127746172080,Общество с ограниченной ответственностью «Союз Святого Иоанна Воина»,http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=913526,"ООО ""СОЮЗ СВ. ИОАННА ВОИНА""",KJIYB JIEOBHTEJIEH HU3KVX LEH\n
193,830443,2021/09/28,2020757263,2020/10/14,7705033512,1027700400771,"Акционерное общество ""СФЕРА""",http://www1.fips.ru/fips_servl/fips_servlet?DB=RUTM&DocNumber=830443,"АО ""СФЕРА""",IMEPKOP\n


__Результаты могли быть и лучше) Совершенству нет предела, возможно, есть более удобоваримый источник исходной информации, или же модель OCR здорового человека, более тонко настроенная, или очень хитрый алгоритм обработки строковых данных, чтобы расшифровать эти иероглифы. В общем, надеюсь продолжить решение этой проблемы в DaData)__